In [2]:
import pandas as pd
import plotly.graph_objects as go

INTERVAL = '5min'

# Загрузка данных из файла
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    return df

# Рассчитать VPOC для каждого временного интервала с фильтрацией по минимальному объему
def calculate_vpoc(df, min_volume=None):
    """
    Вычисляет Volume Point of Control (VPOC) для каждого временного интервала с возможностью фильтрации по минимальному объему.

    Параметры:
        df (pd.DataFrame): Данные с колонками 'timestamp', 'price' и 'size'.
        min_volume (int, optional): Минимальный объем для фильтрации ценовых уровней.

    Возвращает:
        pd.DataFrame: Данные с колонками 'time_bin', 'vpoc_price', 'vpoc_volume'.
    """
    df['time_bin'] = df['timestamp'].dt.floor(INTERVAL)
    vpoc_data = []

    for time_bin, group in df.groupby('time_bin'):
        volume_profile = group.groupby('price')['size'].sum()

        # Применение фильтрации по минимальному объему, если указано
        if min_volume is not None:
            volume_profile = volume_profile[volume_profile >= min_volume]

        if not volume_profile.empty:
            vpoc_price = volume_profile.idxmax()
            vpoc_volume = volume_profile.max()
            vpoc_data.append([time_bin, vpoc_price, vpoc_volume])

    return pd.DataFrame(vpoc_data, columns=['time_bin', 'vpoc_price', 'vpoc_volume'])

# Построить график с использованием Plotly
def plot_vpoc(df, vpoc_df):
    fig = go.Figure()

    # Добавление свечного графика
    df['time_bin'] = df['timestamp'].dt.floor(INTERVAL)
    ohlc = df.groupby('time_bin').agg({
        'price': ['first', 'max', 'min', 'last']
    })
    ohlc.columns = ['open', 'high', 'low', 'close']
    ohlc.reset_index(inplace=True)

    fig.add_trace(go.Candlestick(
        x=ohlc['time_bin'],
        open=ohlc['open'],
        high=ohlc['high'],
        low=ohlc['low'],
        close=ohlc['close'],
        name='OHLC'
    ))

    # Нормализация объемов для определения размеров маркеров
    max_volume = vpoc_df['vpoc_volume'].max()
    min_marker_size = 5   # Минимальный размер маркера
    max_marker_size = 20  # Максимальный размер маркера

    # Функция для масштабирования объема к размеру маркера
    def scale_volume_to_size(volume):
        return ((volume / max_volume) * (max_marker_size - min_marker_size)) + min_marker_size

    # Применение функции масштабирования к столбцу 'vpoc_volume'
    vpoc_df['marker_size'] = vpoc_df['vpoc_volume'].apply(scale_volume_to_size)

    # Добавление VPOC с динамическими размерами маркеров
    fig.add_trace(go.Scatter(
        x=vpoc_df['time_bin'],
        y=vpoc_df['vpoc_price'],
        mode='markers+lines',
        name='VPOC',
        line=dict(color='white', width=0.5),
        marker=dict(
            color='white',
            symbol="circle",
            size=vpoc_df['marker_size'],  # Установка размера маркера
            sizemode='diameter',           # Режим размера маркера
            sizeref=0.68,                     # Коэффициент масштабирования размера маркера
            sizemin=min_marker_size        # Минимальный размер маркера
        )
    ))

    fig.update_layout(
        title=f"BTCUSDT Volume Point of Control ({INTERVAL})",
        xaxis_title='Time',
        yaxis_title='Price',
        template='plotly_dark',
        xaxis_rangeslider_visible=False,
        height=800
    )
        # Фиксация масштаба осей
    fig.update_xaxes(
        fixedrange=True,  # Запретить изменение масштаба по оси X
        range=[ohlc_df['time_bin'].min(), ohlc_df['time_bin'].max()]
    )
    fig.update_yaxes(
        fixedrange=True,  # Запретить изменение масштаба по оси Y
        range=[ohlc_df['low'].min(), ohlc_df['high'].max()]
    )
    fig.show()

# Основной процесс
if __name__ == '__main__':
    file_path = './BTCUSDT2020-05-23.csv'

    # Загрузка данных
    df = load_data(file_path)

    # Расчет VPOC с фильтрацией по минимальному объему
    min_volume = 5  # Установите минимальный объем для фильтрации, или None для отключения фильтрации
    vpoc_df = calculate_vpoc(df, min_volume=min_volume)

    # Построение графика
    plot_vpoc(df, vpoc_df)
